In [31]:
using LinearAlgebra
using TensorKit
using JSON
cd("D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
#push!(LOAD_PATH, "D:\\My Documents\\Code\\Julia_codes\\Tensor network\\IPEPS_TensorKit\\kagome\\SU2_PG")
include("load_tensor.jl")
include("CTM.jl")
#include("my_TensorKit_fun.jl")
#TensorKit.usebraidcache_nonabelian[] = false

D=3
chi=10
tol=1e-6

A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

#load elementary tensor coefficients from json file
function read_string(string)
    nums=ones(length(string))
    for cn=1:length(string)
        nums[cn]=parse.(Float64, split(string[cn])[2])
    end
    return nums
end
json_dict = Dict()
open("LS_D_"*string(D)*"_chi_40.json", "r") do f
    global json_dict
    dicttxt = read(f,String)  # file information to string
    json_dict=JSON.parse(dicttxt)  # parse and transform data
end

Bond_irrep=json_dict["Bond_irrep"]
Triangle_A1_coe=read_string(json_dict["coes"]["Triangle_A1_coe"]["entries"]);
Triangle_A2_coe=read_string(json_dict["coes"]["Triangle_A2_coe"]["entries"]);
if Bond_irrep=="A"
    Bond_A_coe=read_string(json_dict["coes"]["Bond_A_coe"]["entries"]);
elseif Bond_irrep=="B"
    Bond_B_coe=read_string(json_dict["coes"]["Bond_B_coe"]["entries"]);
elseif Bond_irrep=="A+iB"
    Bond_A_coe=read_string(json_dict["coes"]["Bond_A_coe"]["entries"]);
    Bond_B_coe=read_string(json_dict["coes"]["Bond_B_coe"]["entries"]);
end

global triangle_tensor=A1_set[1]*0;
global bond_tensor=A_set[1]*0;
if Bond_irrep=="A"
    global bond_tensor=A_set[1]*0;
    for ct=1:length(Bond_A_coe)
        global bond_tensor=bond_tensor+A_set[ct]*Bond_A_coe[ct];
    end
elseif Bond_irrep=="B"
    global bond_tensor=B_set[1]*0;
    for ct=1:length(Bond_B_coe)
        global bond_tensor=bond_tensor+B_set[ct]*Bond_B_coe[ct];
    end
elseif Bond_irrep=="A+iB"
    global bond_tensor=A_set[1]*0;
    for ct=1:length(Bond_A_coe)
        global bond_tensor=bond_tensor+A_set[ct]*Bond_A_coe[ct];
    end
    for ct=1:length(Bond_B_coe)
        global bond_tensor=bond_tensor+B_set[ct]*Bond_B_coe[ct];
    end
end

for ct=1:length(Triangle_A1_coe)
    global triangle_tensor=triangle_tensor+A1_set[ct]*Triangle_A1_coe[ct];
end
for ct=1:length(Triangle_A2_coe)
    global triangle_tensor=triangle_tensor+im*A2_set[ct]*Triangle_A2_coe[ct];
end

#TensorKit.usebraidcache_nonabelian[] = false
A_set,B_set,A1_set,A2_set, A_set_occu,B_set_occu,A1_set_occu,A2_set_occu, S_label, Sz_label, virtual_particle, Va, Vb=construct_tensor(D);

#load elementary tensor coefficients from json file
function read_string(string)
    nums=ones(length(string))
    for cn=1:length(string)
        nums[cn]=parse.(Float64, split(string[cn])[2])
    end
    return nums
end
json_dict = Dict()
open("LS_D_"*string(D)*"_chi_40.json", "r") do f
    global json_dict
    dicttxt = read(f,String)  # file information to string
    json_dict=JSON.parse(dicttxt)  # parse and transform data
end

Bond_irrep=json_dict["Bond_irrep"]
Triangle_A1_coe=read_string(json_dict["coes"]["Triangle_A1_coe"]["entries"]);
Triangle_A2_coe=read_string(json_dict["coes"]["Triangle_A2_coe"]["entries"]);
if Bond_irrep=="A"
    Bond_A_coe=read_string(json_dict["coes"]["Bond_A_coe"]["entries"]);
elseif Bond_irrep=="B"
    Bond_B_coe=read_string(json_dict["coes"]["Bond_B_coe"]["entries"]);
elseif Bond_irrep=="A+iB"
    Bond_A_coe=read_string(json_dict["coes"]["Bond_A_coe"]["entries"]);
    Bond_B_coe=read_string(json_dict["coes"]["Bond_B_coe"]["entries"]);
end



global triangle_tensor=A1_set[1]*0;
global bond_tensor=A_set[1]*0;
if Bond_irrep=="A"
    global bond_tensor=A_set[1]*0;
    for ct=1:length(Bond_A_coe)
        global bond_tensor=bond_tensor+A_set[ct]*Bond_A_coe[ct];
    end
elseif Bond_irrep=="B"
    global bond_tensor=B_set[1]*0;
    for ct=1:length(Bond_B_coe)
        global bond_tensor=bond_tensor+B_set[ct]*Bond_B_coe[ct];
    end
elseif Bond_irrep=="A+iB"
    global bond_tensor=A_set[1]*0;
    for ct=1:length(Bond_A_coe)
        global bond_tensor=bond_tensor+A_set[ct]*Bond_A_coe[ct];
    end
    for ct=1:length(Bond_B_coe)
        global bond_tensor=bond_tensor+B_set[ct]*Bond_B_coe[ct];
    end
end

for ct=1:length(Triangle_A1_coe)
    global triangle_tensor=triangle_tensor+A1_set[ct]*Triangle_A1_coe[ct];
end
for ct=1:length(Triangle_A2_coe)
    global triangle_tensor=triangle_tensor+im*A2_set[ct]*Triangle_A2_coe[ct];
end

PEPS_tensor=bond_tensor;
@tensor PEPS_tensor[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];

# display(codomain(PEPS_tensor))
# @time PEPS_tensor1= ncon([bond_tensor,bond_tensor,bond_tensor,triangle_tensor,triangle_tensor],[[-1,1,-5],[4,3,-6],[-4,2,-7],[1,3,2],[4,-2,-3]]);

# @time @tensoropt PEPS_tensor2[-1,-2,-3,-4,-5,-6,-7] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];

# @time @tensoropt PEPS_tensor3[:] := bond_tensor[-1,1,-5]*bond_tensor[4,3,-6]*bond_tensor[-4,2,-7]*triangle_tensor[1,3,2]*triangle_tensor[4,-2,-3];


U_phy=unitary(fuse(space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7)), space(PEPS_tensor, 5) ⊗ space(PEPS_tensor, 6) ⊗ space(PEPS_tensor, 7));
@tensor PEPS_tensor[:] :=PEPS_tensor[-1,-2,-3,-4,1,2,3]*U_phy[-5,1,2,3];


# #PEPS_tensor_dense=convert(Array,PEPS_tensor)

# #save initial CTM to compare with other codes
# @time CTM=init_CTM(10,PEPS_tensor,"PBC");
# matwrite("matfile.mat", Dict(
# 	"C1" => convert(Array,CTM["Cset"][1]),
# 	"C2" => convert(Array,CTM["Cset"][2]),
#     "C3" => convert(Array,CTM["Cset"][3]),
#     "C4" => convert(Array,CTM["Cset"][4]),
#     "T1" => convert(Array,CTM["Tset"][1]),
#     "T2" => convert(Array,CTM["Tset"][2]),
#     "T3" => convert(Array,CTM["Tset"][3]),
#     "T4" => convert(Array,CTM["Tset"][4])
# ); compress = false)

type="PBC"
A=PEPS_tensor
display("initialize CTM")
#numind(A)
#numin(A)
#numout(A)
CTM=[]
Cset=Vector(undef,4)
Tset=Vector(undef,4)
#space(A,1)
@time if type=="PBC"
    for direction=1:4    
        inds=(mod1(2-direction,4),mod1(3-direction,4),mod1(4-direction,4),mod1(1-direction,4),5);
        A_rotate=permute(A,inds)
        Ap_rotate=A_rotate'

        @tensor M[:]:=Ap_rotate[1,-1,-3,2,3]*A_rotate[1,-2,-4,2,3];
        Cset[direction]=M;
        @tensor M[:]:=Ap_rotate[-1,-3,-5,1,2]*A_rotate[-2,-4,-6,1,2];
        Tset[direction]=M
    end

    #fuse legs
    ul_set=Vector(undef,4)
    ur_set=Vector(undef,4)
    for direction=1:2
        ul_set[direction]=unitary(fuse(space(Cset[direction], 3) ⊗ space(Cset[direction], 4)), space(Cset[direction], 3) ⊗ space(Cset[direction], 4))
        ur_set[direction]=unitary(fuse(space(Tset[direction], 5) ⊗ space(Tset[direction], 6)), space(Tset[direction], 5) ⊗ space(Tset[direction], 6))
    end
    for direction=3:4
        ul_set[direction]=unitary(fuse(space(Cset[direction], 3) ⊗ space(Cset[direction], 4))', space(Cset[direction], 3) ⊗ space(Cset[direction], 4))
        ur_set[direction]=unitary(fuse(space(Tset[direction], 5) ⊗ space(Tset[direction], 6))', space(Tset[direction], 5) ⊗ space(Tset[direction], 6))
    end
    for direction=1:4
        C=Cset[direction]
        ul=ur_set[mod1(direction-1,4)]
        ur=ul_set[direction]
        ulp=permute(ul',(3,),(1,2,))
        urp=permute(ur',(3,),(1,2,))
        #@tensor Cnew[(-1);(-2)]:=ulp[-1,1,2]*C[1,2,3,4]*ur[-2,3,4]
        @tensor Cnew[:]:=ulp[-1,1,2]*C[1,2,3,4]*ur[-2,3,4]#put all indices in tone side so that its adjoint has the same index order 
        Cset[direction]=Cnew

        T=Tset[direction]
        ul=ul_set[direction]
        ur=ur_set[direction]
        ulp=permute(ul',(3,),(1,2,))
        urp=permute(ur',(3,),(1,2,))
        #@tensor Tnew[(-1);(-2,-3,-4)]:=ulp[-1,1,2]*T[1,2,-2,-3,3,4]*ur[-4,3,4]
        @tensor Tnew[:]:=ulp[-1,1,2]*T[1,2,-2,-3,3,4]*ur[-4,3,4]#put all indices in tone side so that its adjoint has the same index order 
        Tset[direction]=Tnew
    end
elseif type=="random"
end
CTM=Dict([("Cset", Cset), ("Tset", Tset)])

# matwrite("matfile.mat", Dict(
# 	"C1" => convert(Array,CTM["Cset"][1]),
# 	"C2" => convert(Array,CTM["Cset"][2]),
#     "C3" => convert(Array,CTM["Cset"][3]),
#     "C4" => convert(Array,CTM["Cset"][4]),
#     "T1" => convert(Array,CTM["Tset"][1]),
#     "T2" => convert(Array,CTM["Tset"][2]),
#     "T3" => convert(Array,CTM["Tset"][3]),
#     "T4" => convert(Array,CTM["Tset"][4])
# ); compress = false)


"initialize CTM"

  0.102881 seconds (167.52 k allocations: 11.415 MiB, 92.99% compilation time)


Dict{String, Vector{Any}} with 2 entries:
  "Cset" => [TensorMap((Rep[SU₂](0=>2, 1/2=>2, 1=>1) ⊗ Rep[SU₂](0=>2, 1/2=>2, 1…
  "Tset" => [TensorMap((Rep[SU₂](0=>2, 1/2=>2, 1=>1)' ⊗ Rep[SU₂](0=>1, 1/2=>1) …

In [32]:
function build_double_layer(A,operator)
    #display(space(A))
    A=permute(A,(1,2,),(3,4,5));
    U_L=unitary(fuse(space(A, 1)' ⊗ space(A, 1)), space(A, 1)' ⊗ space(A, 1));
    U_D=unitary(fuse(space(A, 2)' ⊗ space(A, 2)), space(A, 2)' ⊗ space(A, 2));
    U_R=inv(U_L);
    U_U=inv(U_D);
    # display(space(U_L))
    # display(space(U_D))
    # display(space(U_R))
    # display(space(U_D))

    uM,sM,vM=tsvd(A);
    uM=uM*sM
    display(space(uM))
    display(space(vM))
    
    uM=permute(uM,(1,2,3,),())
    @tensor double_LD[:]:=uM'[-1,-3,-5]*uM[-2,-4,-6];
    display(space(double_LD))
    
    vM=permute(vM,(1,2,3,4,),());
    if isempty(operator)
        @tensor double_RU[:]:=vM'[-1,-3,-5,1]*vM[-2,-4,-6,1];
    else
        @tensor double_RU[:]:=vM'[-1,-3,-5,1]*vM[-2,-4,-6,2]*operator[1,2];
    end
    display(space(double_RU))

    double_LD=permute(double_LD,(1,2,),(3,4,5,6,));
    double_LD=U_L*double_LD;
    double_LD=permute(double_LD,(2,3,),(1,4,5,));
    double_LD=U_D*double_LD;
    double_LD=permute(double_LD,(2,1,),(3,4,));
    #display(space(double_LD))
    double_RU=permute(double_RU,(1,2,5,6,),(3,4,));
    double_RU=double_RU*U_R;
    double_RU=permute(double_RU,(1,2,5,),(3,4,));
    double_RU=double_RU*U_U;
    double_LD=permute(double_LD,(1,2,),(3,4,));
    double_RU=permute(double_RU,(1,2,),(3,4,));
    AA_fused=double_LD*double_RU;
    
    return AA_fused, U_L,U_D,U_R,U_U
end


build_double_layer (generic function with 1 method)

In [33]:
AA_fused, U_L,U_D,U_R,U_U=build_double_layer(A,[]);

(Rep[SU₂](0=>1, 1/2=>1) ⊗ Rep[SU₂](0=>1, 1/2=>1)') ← Rep[SU₂](0=>2, 1/2=>2, 1=>1)

Rep[SU₂](0=>2, 1/2=>2, 1=>1) ← (Rep[SU₂](0=>1, 1/2=>1) ⊗ Rep[SU₂](0=>1, 1/2=>1)' ⊗ Rep[SU₂](1/2=>2, 3/2=>1)')

(Rep[SU₂](0=>1, 1/2=>1)' ⊗ Rep[SU₂](0=>1, 1/2=>1) ⊗ Rep[SU₂](0=>1, 1/2=>1) ⊗ Rep[SU₂](0=>1, 1/2=>1)' ⊗ Rep[SU₂](0=>2, 1/2=>2, 1=>1) ⊗ Rep[SU₂](0=>2, 1/2=>2, 1=>1)') ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

(Rep[SU₂](0=>2, 1/2=>2, 1=>1)' ⊗ Rep[SU₂](0=>2, 1/2=>2, 1=>1) ⊗ Rep[SU₂](0=>1, 1/2=>1) ⊗ Rep[SU₂](0=>1, 1/2=>1)' ⊗ Rep[SU₂](0=>1, 1/2=>1)' ⊗ Rep[SU₂](0=>1, 1/2=>1)) ← ProductSpace{GradedSpace{SU2Irrep, TensorKit.SortedVectorDict{SU2Irrep, Int64}}, 0}()

In [34]:
function fuse_CTM_legs(CTM,U_L,U_D,U_R,U_U)
    #fuse CTM legs
    Tset=CTM["Tset"];

    #T4
    T4=permute(Tset[4],(1,4,),(2,3,));
    T4=T4*U_R;
    T4=permute(T4,(1,3,2,),());
    Tset[4]=T4
    #display(space(T4))

    #T3
    T3=permute(Tset[3],(1,4,),(2,3,));
    T3=T3*U_U;
    T3=permute(T3,(1,3,2,),());
    Tset[3]=T3
    #display(space(T3))

    #T2
    T2=permute(Tset[2],(2,3,),(1,4,));
    T2=U_L*T2;
    T2=permute(T2,(2,1,3,),());
    Tset[2]=T2
    #display(space(T2))

    #T1
    T1=permute(Tset[1],(2,3,),(1,4,));
    T1=U_D*T1;
    T1=permute(T1,(2,1,3,),());
    Tset[1]=T1
    #display(space(T1))

    CTM["Tset"]=Tset;
    return CTM
end


fuse_CTM_legs (generic function with 1 method)

In [35]:
CTM=fuse_CTM_legs(CTM,U_L,U_D,U_R,U_U);

In [36]:
function CTM_ite_test(Cset, Tset, AA_fused, chi, direction)
    Tset=CTM["Tset"];
    Cset=CTM["Cset"];

    AA=permute(AA_fused, (mod1(2-direction,4),mod1(3-direction,4),mod1(4-direction,4),mod1(1-direction,4),),());
                
    @tensor MMup[:]:=Cset[mod1(direction,4)][1,2]*Tset[mod1(direction,4)][2,3,-3]*Tset[mod1(direction-1,4)][-1,4,1]*AA[4,-2,-4,3]; 
    @tensor MMlow[:]:=Tset[mod1(direction-1,4)][1,3,-1]*AA[3,4,-4,-2]*Cset[mod1(direction-1,4)][2,1]*Tset[mod1(direction-2,4)][-3,4,2]; 

    @tensor MMup_reflect[:]:=Tset[mod1(direction,4)][-1,3,1]* Cset[mod1(direction+1,4)][1,2]* AA[-2,-4,4,3]* Tset[mod1(direction+1,4)][2,4,-3];
    #@tensor MMlow_reflect[:]:=AA[-2,4,3,-4]*Tset[mod1(direction+1,4)][-3,3,1]*Tset[mod1(direction-2,4)][2,4,-1]*Cset[mod1(direction-2,4)][1,2]; #this is slow compared to other coners, I don't know why            
    @tensor MMlow_reflect[:]:=Tset[mod1(direction+1,4)][-4,-3,2]*Tset[mod1(direction-2,4)][1,-2,-1]*Cset[mod1(direction-2,4)][2,1]; 
    @tensor MMlow_reflect[:]:=MMlow_reflect[-1,1,2,-3]*AA[-2,1,2,-4]; 

    MMup=permute(MMup,(1,2,),(3,4,))

    # _,ss,_=tsvd(MMup)
    # display(convert(Array,ss))

    MMlow=permute(MMlow,(1,2,),(3,4,))
    MMup_reflect=permute(MMup_reflect,(1,2,),(3,4,))
    MMlow_reflect=permute(MMlow_reflect,(1,2,),(3,4,))
    _, RMup=leftorth(permute(MMup*MMup_reflect,(3,4,),(1,2,)));
    _, RMlow=leftorth(permute(MMlow*MMlow_reflect,(3,4,),(1,2,))); 


    RMlow=permute(RMlow,(2,3,),(1,));
    M=RMup*RMlow;



    uM,sM,vM = tsvd(M; trunc=truncdim(chi));
    #uM,sM,vM = tsvd(M; trunc=trunbelow(1e-34)); #this looks to truncate absolute value, not relative value
    #display(convert(Array,sM))
    #display(convert(Array,inv(sM)))
    sM=sM/norm(sM)
    sM_inv=inv(sM);
    sM_dense=convert(Array,sM)
    #display(sM_dense)
    trun_tol=1e-12
    for c1=1:size(sM_dense,1)
        if sM_dense[c1,c1]<trun_tol
            sM_dense[c1,c1]=0;
        end
    end
    #display(sM_dense)
    #display(pinv.(sM_dense))

    #display(sM_inv)
    #display(convert(Array,sM_inv))
    sM_inv_sqrt=sqrt.(convert(Array,sM_inv))
    #display(space(sM_inv))
    #display(sM_inv_sqrt)
    sM_inv_sqrt=TensorMap(pinv.(sqrt.(sM_dense)),codomain(sM_inv)←domain(sM_inv))

    PM_inv=RMlow*vM'*sM_inv_sqrt; 
    PM=sM_inv_sqrt*uM'*RMup; 
    PM=permute(PM,(2,3,),(1,));

    @tensor M5tem[:]:=Tset[mod1(direction-1,4)][4,3,1]*AA[3,5,-2,2]* PM_inv[4,5,-1]* PM[1,2,-3];
    @tensor M1tem[:]:=Cset[mod1(direction,4)][1,2]*Tset[mod1(direction,4)][2,3,-2]*PM_inv[1,3,-1];
    @tensor M7tem[:]:=Cset[mod1(direction-1,4)][1,2]*Tset[mod1(direction-2,4)][-1,3,1]* PM[2,3,-2];

    Cset[mod1(direction,4)]=M1tem/norm(M1tem);
    Tset[mod1(direction-1,4)]=M5tem/norm(M5tem);
    Cset[mod1(direction-1,4)]=M7tem/norm(M7tem);
    return Cset,Tset
end

CTM_ite_test (generic function with 1 method)

In [37]:

    #Ref: PHYSICAL REVIEW B 98, 235148 (2018)
    #initial corner transfer matrix

    Cset=CTM["Cset"];
    Tset=CTM["Tset"];
    conv_check="singular_value"

    ss_old=ones(chi)*2;
    d=2;
    rho_old=Matrix(I,d^3,d^3);

    #Iteration
    CTM_ite_nums=50;
    display("start CTM iterations:")
    for ci=1:CTM_ite_nums
        direction_order=[1,2,3,4];
        @time for direction in direction_order
            Cset,Tset=CTM_ite_test(Cset, Tset, AA_fused, chi, direction);
        end
        if conv_check=="singular_value" #check convergence of singular value
            _,ss_new,_=svd(permute(Cset[1],(1,),(2,)));
            ss_new=convert(Array,ss_new);
            ss_new=sort(diag(ss_new), rev=true);
            ss_old=ss_old/ss_old[1];
            ss_new=ss_new/ss_new[1];
            display(ss_new)
            if length(ss_old)>length(ss_new)
                dss=copy(ss_old);
                siz=length(ss_new)
            elseif length(ss_old)<=length(ss_new)
                dss=copy(ss_new);
                siz=length(ss_old)
            end
            dss[1:siz]=ss_old[1:siz]-ss_new[1:siz]
            er=norm(dss);
            display("CTMRG iteration: "*string(ci)*", CTMRG err: "*string(er));
            if er<tol
                break;
            end
            ss_old=ss_new;
        elseif conv_check=="density_matrix" #check reduced density matrix
            
            # ob_opts.SiteNumber=1;
            # CTM_tem.Cset=Cset;
            # CTM_tem.Tset=Tset;
            # rho_new=ob_CTMRG(CTM_tem,A,ob_opts).A;
            # er=sum(sum((abs(rho_old-rho_new))));
            # disp(['CTMRG iteration: ',num2str(ci),' CTMRG err: ',num2str(er)]);
            # if er<tol
            #     break;
            # end
            # rho_old=rho_new;
        end
        
        # if ci==CTM_ite_nums
        #     display(er)
        #     warn("CTMRG does not converge: " * string(er));
        # end
    end






    

"start CTM iterations:"

10-element Vector{Float64}:
 1.0
 0.37667701357582734
 0.37667701357582734
 0.37667701357581324
 0.37667701357581324
 0.3609652764438206
 0.20514117994886827
 0.20514117994886827
 0.20514117994886136
 0.20514117994886136

"CTMRG iteration: 1, CTMRG err: 2.118889741062967"

10-element Vector{Float64}:
 1.0
 0.5169463664848383
 0.3791198054039318
 0.3791198054039318
 0.3791198054039311
 0.3791198054039311
 0.2524616900975488
 0.2524616900975488
 0.2524616900975476
 0.2524616900975476

"CTMRG iteration: 2, CTMRG err: 0.170234845745368"

...

10-element Vector{Float64}:
 1.0
 0.40909079133340787
 0.40909079133340787
 0.40909024060289034
 0.40909024060289034
 0.15912450771731068
 0.047349476931181876
 0.047349476931181876
 0.04734943755122394
 0.04734943755122394

"CTMRG iteration: 19, CTMRG err: 1.989125885369942e-6"

10-element Vector{Float64}:
 1.0
 0.40909041797357615
 0.40909041797357615
 0.40909027163100137
 0.40909027163100137
 0.15912439497248132
 0.04734940626709752
 0.04734940626709752
 0.04734939564922307
 0.04734939564922307

"CTMRG iteration: 20, CTMRG err: 5.540129897090912e-7"

  0.120845 seconds (298.02 k allocations: 22.031 MiB, 79.76% compilation time)
  0.019235 seconds (133.19 k allocations: 11.379 MiB)
  0.023393 seconds (128.26 k allocations: 10.217 MiB)
  0.045900 seconds (128.26 k allocations: 10.217 MiB, 49.83% gc time)
  0.019606 seconds (128.26 k allocations: 10.217 MiB)
  0.019817 seconds (128.26 k allocations: 10.217 MiB)
  0.021650 seconds (128.26 k allocations: 10.217 MiB)
  0.038540 seconds (146.85 k allocations: 11.731 MiB, 46.19% compilation time)
  0.022824 seconds (130.51 k allocations: 11.235 MiB)
  0.049570 seconds (125.27 k allocations: 10.078 MiB, 45.84% gc time)
  0.022908 seconds (124.53 k allocations: 9.907 MiB)
  0.027956 seconds (124.53 k allocations: 9.907 MiB)
  0.022822 seconds (124.53 k allocations: 9.907 MiB)
  0.019609 seconds (131.58 k allocations: 10.845 MiB)
  0.027414 seconds (128.26 k allocations: 10.217 MiB)
  0.028871 seconds (128.26 k allocations: 10.217 MiB)
  0.067992 seconds (128.26 k allocations: 10.217 MiB, 42.